In [1]:
# data目錄
base_dir = "D:\\artificial Intelligence Research\\mango\\mango_new_data\\original"

# model parameters
depth=20
attention_module = 'se_block' # (cbam_block / se_block / None)

# Training parameters
target_size=(256,256)
batch_size = 16
epochs=100

In [2]:
# image preprocessing
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   fill_mode='nearest')

valid_datagen = ImageDataGenerator(rescale=1./255)



training_set = train_datagen.flow_from_directory(base_dir+'/Train',
                                                 target_size=target_size,
                                                 batch_size=batch_size,
                                                 class_mode='categorical')

valid_set = valid_datagen.flow_from_directory(base_dir+'/Dev',
                                            target_size=target_size,
                                            batch_size=batch_size,
                                            class_mode='categorical')

train_num = training_set.samples
valid_num = valid_set.samples

Using TensorFlow backend.


Found 45000 images belonging to 3 classes.
Found 7000 images belonging to 3 classes.


In [3]:
#熱重啟
from keras.callbacks import Callback
import keras.backend as K
import numpy as np

class SGDRScheduler(Callback):

    def __init__(self,
                 min_lr,
                 max_lr,
                 steps_per_epoch,
                 lr_decay=1,
                 cycle_length=10,
                 mult_factor=2):

        self.min_lr = min_lr
        self.max_lr = max_lr
        self.lr_decay = lr_decay

        self.batch_since_restart = 0
        self.next_restart = cycle_length

        self.steps_per_epoch = steps_per_epoch

        self.cycle_length = cycle_length
        self.mult_factor = mult_factor

        self.history = {}

    def clr(self):
        '''Calculate the learning rate.'''
        fraction_to_restart = self.batch_since_restart / (self.steps_per_epoch * self.cycle_length)
        lr = self.min_lr + 0.5 * (self.max_lr - self.min_lr) * (1 + np.cos(fraction_to_restart * np.pi))
        return lr

    def on_train_begin(self, logs={}):
        '''Initialize the learning rate to the minimum value at the start of training.'''
        logs = logs or {}
        K.set_value(self.model.optimizer.lr, self.max_lr)

    def on_batch_end(self, batch, logs={}):
        '''Record previous batch statistics and update the learning rate.'''
        logs = logs or {}
        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        self.batch_since_restart += 1
        K.set_value(self.model.optimizer.lr, self.clr())

    def on_epoch_end(self, epoch, logs={}):
        '''Check for end of current cycle, apply restarts when necessary.'''
        if epoch + 1 == self.next_restart:
            self.batch_since_restart = 0
            self.cycle_length = np.ceil(self.cycle_length * self.mult_factor)
            self.next_restart += self.cycle_length
            self.max_lr *= self.lr_decay
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs={}):
        '''Set weights to the values from the end of the most recent cycle for best performance.'''
        self.model.set_weights(self.best_weights)

In [4]:
#熱重啟參數
SGDR = SGDRScheduler(min_lr=1e-5,
                         max_lr=1e-2,
                         steps_per_epoch=np.ceil(train_num/batch_size),
                         lr_decay=0.9,
                         cycle_length=5,
                         mult_factor=1.5)

In [5]:
# build model
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation,GlobalAveragePooling2D
from keras.layers import AveragePooling2D, Input, Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
import os
from keras.models import Sequential
from models import resnet_v1,resnet_v2

model = Sequential()

model_resnet =resnet_v1.resnet_v1(input_shape=(256,256,3),depth=depth,num_classes=3,attention_module=attention_module)

model.add(model_resnet)


base_model = f'resnet{depth}'
model_type = base_model if attention_module==None else base_model+'_'+attention_module
print("model_type:",model_type)

model.summary()

model_type: resnet20_se_block
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_1 (Model)              (None, 3)                 290493    
Total params: 290,493
Trainable params: 289,117
Non-trainable params: 1,376
_________________________________________________________________


In [6]:
from tensorflow.keras import optimizers
#sgd = optimizers.SGD(lr=8e-3)
model.compile(optimizer="SGD",
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])



In [7]:
from keras.callbacks import EarlyStopping
# early stopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=25)

train_num = training_set.samples
valid_num = valid_set.samples
history0 = model.fit_generator(training_set,
                    steps_per_epoch=train_num//batch_size,
                    validation_data=valid_set,
                    epochs=epochs,
                    validation_steps=valid_num//batch_size,
                    callbacks=[SGDR])

Epoch 1/100
2812/2812 [==============================] - 1679s 597ms/step - loss: 1.0030 - categorical_accuracy: 0.6870 - val_loss: 1.0581 - val_categorical_accuracy: 0.8018
Epoch 2/100
2812/2812 [==============================] - 1673s 595ms/step - loss: 0.8704 - categorical_accuracy: 0.7150 - val_loss: 0.7548 - val_categorical_accuracy: 0.7871
Epoch 3/100
 929/2812 [========>.....................] - ETA: 17:58 - loss: 0.8432 - categorical_accuracy: 0.7281

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

acc = history0.history['categorical_accuracy']
val_acc = history0.history['val_categorical_accuracy']
loss = history0.history['loss']
val_loss = history0.history['val_loss']
epochs = range(1, len(loss) + 1)

x=0
y=0
for i in val_acc[-(len(epochs)//5):]:
    x+=1
    y+=i
print(y/x)

#accuracy plot
plt.plot(epochs, acc, color='green', label='Training Accuracy')
plt.plot(epochs, val_acc, color='blue', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.figure()
#loss plot
plt.plot(epochs, loss, color='pink', label='Training Loss')
plt.plot(epochs, val_loss, color='red', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()